In [4]:
import tensorflow as tf
with tf.compat.v1.Session() as sess:
    hello = tf.constant('Hello, Tensorflow!')
    print(sess.run(hello))

b'Hello, Tensorflow!'


In [5]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [6]:
global uniq_train_words
uniq_train_words=set()


In [7]:
from sklearn.datasets import load_files
# Read training files
reviews_set = load_files("C:/Users/admin/Desktop/data/")
# Lets get training reviews and training labels in sepearate lists
reviews, labels = reviews_set.data, reviews_set.target

# Let's understand the two lists: reviews (text_train) and their labels (y_train)
print("Data type of reviews: ",type(reviews))
print("Total number of reviews: ",len(reviews))
print("Text at index 6:\n ", reviews[6])
print("Label of the review at Index 6: ",labels[6])
# 0 for negative and 1 for positive

Data type of reviews:  <class 'list'>
Total number of reviews:  2003
Text at index 6:
  b"It's a colorful slasher movie. That's about it.<br /><br />It has the mystery element that SCREAM made so popular in slasher movies, but I never care for such things. Figuring out who's the bad guy is not that interesting considering the clues are all misleading anyway.<br /><br />The death scenes were inventive and gorey, bringing back memories of 80's horror movies like Friday the 13th. <br /><br />Another nice thing about this movie is that it's hard to pinpoint the surviving girl, unlike in SCREAM and IKWYDLS where it was obvious. <br /><br />People who don't like slasher movies won't like this movie. As simple as that. I truly enjoyed it and I plan to watch it again while waiting for more of the same. <br /><br />--MB"
Label of the review at Index 6:  1


In [8]:
train_rev, test_rev,train_lbls, test_lbls = train_test_split(reviews, labels, test_size=0.33)

In [9]:
import nltk
import string
from nltk.stem import PorterStemmer

#Custom function for text noise removal and stemming
def noise_removal(text_data, isTraining=True):
    
    stemmer = PorterStemmer()
    punctuation_list=list(string.punctuation)
    # additional punctuation marks
    punctuation_list.append("--") 
    punctuation_list.append("''")
    punctuation_list.append("``")
    text_data=text_data.replace("<br />", " ") 
    # some  stop words
    stop_words=['a','an','he','she','it','am','will','have','has','i','you','me','\'s','``','\'','(',')','*****','...']
    noise_list=punctuation_list +stop_words
    # making sure in testing we are keeping only words in training and removing noise in both training and testing
    #first tokenize, if not in noise_list, stem
    filtered_words=[stemmer.stem(w) for w in nltk.word_tokenize(text_data) if w not in noise_list]
       
    if isTraining==True:
        for w in filtered_words:
            uniq_train_words.add(w )
    else:# during testing
        filtered_words=[w for w in filtered_words if w in uniq_train_words]
        
    return  " ".join(filtered_words) # convert the list of tokenize words to a string

In [11]:
max_length = 300

In [12]:
def pre_process(reviews,isTraining=True):
    reviews= [noise_removal(text.decode(),isTraining) for text in reviews]
    # Integer encode the documents
    vocab_size=len(uniq_train_words)
    print(vocab_size)
    
    print (reviews[0])
    encoded_reviews = [one_hot(review, vocab_size) for review in reviews]
    
    # pad documents to a max length of n words
    padded_reviews = pad_sequences(encoded_reviews, maxlen=max_length, padding='post')
    return padded_reviews